In [1]:
#include "kalman.hpp"

In [2]:
#include <iostream>
#include <stdexcept>
#include <vector>


In [3]:
__global__ void matAddKernel(double* a, double* b, double* c, int rows, int cols) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < cols && row < rows) {
        int idx = row * cols + col;
        c[idx] = a[idx] + b[idx];
    }
}

In [4]:
__global__ void matSubKernel(double* a, double* b, double* c, int rows, int cols) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < cols && row < rows) {
        int idx = row * cols + col;
        c[idx] = a[idx] - b[idx];
    }
}

In [5]:
__global__ void matTransposeKernel(double* a, double* c, int rows, int cols) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    if (col < cols && row < rows) {
        int idx_in = row * cols + col;
        int idx_out = col * rows + row;
        c[idx_out] = a[idx_in];
    }
}

In [6]:
__global__ void matvecmulKernel(double* d_mat, double* d_vec, double* d_result, int rows, int cols) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < rows) {
        double sum = 0.0;
        for (int j = 0; j < cols; j++) {
            sum += d_mat[tid * cols + j] * d_vec[j];
        }
        d_result[tid] = sum;
    }
}

In [7]:
__global__ void matmulKernel(double* d_a, double* d_b, double* d_result, int rowsA, int colsA, int colsB) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < rowsA && col < colsB) {
        double value = 0.0;
        for (int k = 0; k < colsA; k++) {
            value += d_a[row * colsA + k] * d_b[k * colsB + col];
        }
        d_result[row * colsB + col] = value;
    }
}

In [8]:
__global__ void vecsubKernel(const double* a, const double* b, double* result, int len) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < len) {
        result[idx] = a[idx] - b[idx];
    }
}

In [9]:
std::vector<std::vector<double>> mataddCUDA(const std::vector<std::vector<double>>& a, const std::vector<std::vector<double>>& b) {
    int rows = a.size();
    int cols = a[0].size();
    
    double* h_a = new double[rows*cols];
    double* h_b = new double[rows*cols];
    double* h_c = new double[rows*cols];
    
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            h_a[i*cols + j] = a[i][j];
            h_b[i*cols + j] = b[i][j];
        }
    }
    
    double* d_a, * d_b, * d_c;
    cudaMalloc((void**)&d_a, rows*cols*sizeof(double));
    cudaMalloc((void**)&d_b, rows*cols*sizeof(double));
    cudaMalloc((void**)&d_c, rows*cols*sizeof(double));

    cudaMemcpy(d_a, h_a, rows*cols*sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, rows*cols*sizeof(double), cudaMemcpyHostToDevice);

    dim3 dimBlock(16, 16);
    dim3 dimGrid((cols + dimBlock.x - 1) / dimBlock.x, (rows + dimBlock.y - 1) / dimBlock.y);
    matAddKernel<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, rows, cols);

    cudaMemcpy(h_c, d_c, rows*cols*sizeof(double), cudaMemcpyDeviceToHost);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    std::vector<std::vector<double>> result(rows, std::vector<double>(cols));
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[i][j] = h_c[i*cols + j];
        }
    }

    delete[] h_a;
    delete[] h_b;
    delete[] h_c;

    return result;
}

In [10]:
std::vector<std::vector<double>> matsubCUDA(const std::vector<std::vector<double>>& a, const std::vector<std::vector<double>>& b) {
    int rows = a.size();
    int cols = a[0].size();
    
    double* h_a = new double[rows*cols];
    double* h_b = new double[rows*cols];
    double* h_c = new double[rows*cols];
    
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            h_a[i*cols + j] = a[i][j];
            h_b[i*cols + j] = b[i][j];
        }
    }

    double* d_a, * d_b, * d_c;
    cudaMalloc((void**)&d_a, rows*cols*sizeof(double));
    cudaMalloc((void**)&d_b, rows*cols*sizeof(double));
    cudaMalloc((void**)&d_c, rows*cols*sizeof(double));

    cudaMemcpy(d_a, h_a, rows*cols*sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, rows*cols*sizeof(double), cudaMemcpyHostToDevice);

    dim3 dimBlock(16, 16);  
    dim3 dimGrid((cols + dimBlock.x - 1) / dimBlock.x, (rows + dimBlock.y - 1) / dimBlock.y);
    matSubKernel<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, rows, cols);

    cudaMemcpy(h_c, d_c, rows*cols*sizeof(double), cudaMemcpyDeviceToHost);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    std::vector<std::vector<double>> result(rows, std::vector<double>(cols));
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[i][j] = h_c[i*cols + j];
        }
    }

    delete[] h_a;
    delete[] h_b;
    delete[] h_c;

    return result;
}

In [11]:
std::vector<std::vector<double>> mattransposeCUDA(const std::vector<std::vector<double>>& a) {
    int rows = a.size();
    int cols = a[0].size();
    
    double* h_a = new double[rows*cols];
    double* h_c = new double[cols*rows];
    
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            h_a[i*cols + j] = a[i][j];
        }
    }

    double* d_a, * d_c;
    cudaMalloc((void**)&d_a, rows*cols*sizeof(double));
    cudaMalloc((void**)&d_c, cols*rows*sizeof(double));

    cudaMemcpy(d_a, h_a, rows*cols*sizeof(double), cudaMemcpyHostToDevice);

    dim3 dimBlock(16, 16);
    dim3 dimGrid((cols + dimBlock.x - 1) / dimBlock.x, (rows + dimBlock.y - 1) / dimBlock.y);
    matTransposeKernel<<<dimGrid, dimBlock>>>(d_a, d_c, rows, cols);

    cudaMemcpy(h_c, d_c, cols*rows*sizeof(double), cudaMemcpyDeviceToHost);

    cudaFree(d_a);
    cudaFree(d_c);

    std::vector<std::vector<double>> result(cols, std::vector<double>(rows));
    for (int i = 0; i < cols; i++) {
        for (int j = 0; j < rows; j++) {
            result[i][j] = h_c[i*rows + j];
        }
    }

    delete[] h_a;
    delete[] h_c;

    return result;
}

In [12]:
// helper function: matrix inversion
std::vector<std::vector<double>> matinverse(const std::vector<std::vector<double>>& a) {
    size_t n = a.size();
    
    if (n != a[0].size()) {
        std::cout<<" Shape of a : "<<a.size()<<","<<a[0].size()<<"\n";
        throw std::runtime_error("Only square matrices are supported for inversion.");
    }

    // Handle 1x1 matrix
    if (n == 1) {
        if (a[0][0] == 0) {
            throw std::runtime_error("singular matrix");
        }
        return {{1.0 / a[0][0]}};
    }
    
    if (n == 2) {
        double determinant = a[0][0] * a[1][1] - a[0][1] * a[1][0];
        if (determinant == 0) {
            throw std::runtime_error("singular matrix");
        }

        std::vector<std::vector<double>> result(2, std::vector<double>(2));
        result[0][0] = a[1][1] / determinant;
        result[0][1] = -a[0][1] / determinant;
        result[1][0] = -a[1][0] / determinant;
        result[1][1] = a[0][0] / determinant;

        return result;
    }

    if (n == 3) {
        double determinant = a[0][0]*(a[1][1]*a[2][2]-a[2][1]*a[1][2]) 
                             - a[0][1]*(a[1][0]*a[2][2]-a[1][2]*a[2][0]) 
                             + a[0][2]*(a[1][0]*a[2][1]-a[1][1]*a[2][0]);
        
        if (determinant == 0) {
            throw std::runtime_error("singular matrix");
        }

        std::vector<std::vector<double>> result(3, std::vector<double>(3));

        result[0][0] = (a[1][1] * a[2][2] - a[2][1] * a[1][2]) / determinant;
        result[0][1] = (a[0][2] * a[2][1] - a[0][1] * a[2][2]) / determinant;
        result[0][2] = (a[0][1] * a[1][2] - a[0][2] * a[1][1]) / determinant;
        result[1][0] = (a[1][2] * a[2][0] - a[1][0] * a[2][2]) / determinant;
        result[1][1] = (a[0][0] * a[2][2] - a[0][2] * a[2][0]) / determinant;
        result[1][2] = (a[1][0] * a[0][2] - a[0][0] * a[1][2]) / determinant;
        result[2][0] = (a[1][0] * a[2][1] - a[2][0] * a[1][1]) / determinant;
        result[2][1] = (a[2][0] * a[0][1] - a[0][0] * a[2][1]) / determinant;
        result[2][2] = (a[0][0] * a[1][1] - a[1][0] * a[0][1]) / determinant;

        return result;
    }

    throw std::runtime_error("Only 2x2 and 3x3 matrices supported for inversion");
}



In [13]:
std::vector<double> matvecmulCUDA(const std::vector<std::vector<double>>& a, const std::vector<double>& b) {
    int rows = a.size();
    int cols = a[0].size();

    if (cols != b.size()) {
        throw std::runtime_error("Matrix dims do not match for multiplication.");
    }

    std::vector<double> h_mat(rows * cols);
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            h_mat[i * cols + j] = a[i][j];
        }
    }

    double *d_mat, *d_vec, *d_result;
    cudaMalloc((void**)&d_mat, rows * cols * sizeof(double));
    cudaMalloc((void**)&d_vec, cols * sizeof(double));
    cudaMalloc((void**)&d_result, rows * sizeof(double));

    cudaMemcpy(d_mat, h_mat.data(), rows * cols * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_vec, b.data(), cols * sizeof(double), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int gridSize = (rows + blockSize - 1) / blockSize;

    matvecmulKernel<<<gridSize, blockSize>>>(d_mat, d_vec, d_result, rows, cols);

    std::vector<double> result(rows);
    cudaMemcpy(result.data(), d_result, rows * sizeof(double), cudaMemcpyDeviceToHost);

    cudaFree(d_mat);
    cudaFree(d_vec);
    cudaFree(d_result);

    return result;
}

In [14]:
std::vector<std::vector<double>> matmulCUDA(const std::vector<std::vector<double>>& a, const std::vector<std::vector<double>>& b) {
    int rowsA = a.size();
    int colsA = a[0].size();
    int rowsB = b.size();
    int colsB = b[0].size();

    if (colsA != rowsB) {
        throw std::runtime_error("Matrix dims do not match for multiplication.");
    }

    std::vector<double> h_a(rowsA * colsA);
    std::vector<double> h_b(rowsB * colsB);

    for (int i = 0; i < rowsA; i++) {
        for (int j = 0; j < colsA; j++) {
            h_a[i * colsA + j] = a[i][j];
        }
    }

    for (int i = 0; i < rowsB; i++) {
        for (int j = 0; j < colsB; j++) {
            h_b[i * colsB + j] = b[i][j];
        }
    }

    double *d_a, *d_b, *d_result;
    
    cudaMalloc((void**)&d_a, rowsA * colsA * sizeof(double));
    cudaMalloc((void**)&d_b, rowsB * colsB * sizeof(double));
    cudaMalloc((void**)&d_result, rowsA * colsB * sizeof(double));

    cudaMemcpy(d_a, h_a.data(), rowsA * colsA * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b.data(), rowsB * colsB * sizeof(double), cudaMemcpyHostToDevice);

    dim3 blockSize(16, 16);
    dim3 gridSize((colsB + blockSize.x - 1) / blockSize.x, (rowsA + blockSize.y - 1) / blockSize.y);

    matmulKernel<<<gridSize, blockSize>>>(d_a, d_b, d_result, rowsA, colsA, colsB);

    std::vector<double> h_result(rowsA * colsB);
    cudaMemcpy(h_result.data(), d_result, rowsA * colsB * sizeof(double), cudaMemcpyDeviceToHost);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_result);
    
    std::vector<std::vector<double>> result(rowsA, std::vector<double>(colsB));
    for (int i = 0; i < rowsA; i++) {
        for (int j = 0; j < colsB; j++) {
            result[i][j] = h_result[i * colsB + j];
        }
    }

    return result;
}

In [15]:
std::vector<double> vecsubCUDA(const std::vector<double>& a, const std::vector<double>& b) {
    int len = a.size();

    double* d_a;
    double* d_b;
    double* d_result;

    cudaMalloc((void**)&d_a, len * sizeof(double));
    cudaMalloc((void**)&d_b, len * sizeof(double));
    cudaMalloc((void**)&d_result, len * sizeof(double));

    cudaMemcpy(d_a, a.data(), len * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b.data(), len * sizeof(double), cudaMemcpyHostToDevice);

    int blockSize = 256; 
    int gridSize = (len + blockSize - 1) / blockSize;

    vecsubKernel<<<gridSize, blockSize>>>(d_a, d_b, d_result, len);

    std::vector<double> result(len);
    cudaMemcpy(result.data(), d_result, len * sizeof(double), cudaMemcpyDeviceToHost);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_result);

    return result;
}


In [16]:
void printMatrix(const std::vector<std::vector<double>>& matrix) {
    for (size_t i = 0; i < matrix.size(); i++) {
        for (size_t j = 0; j < matrix[i].size(); j++) {
            std::cout << matrix[i][j] << " ";
        }
        std::cout << std::endl;
    }
}

In [17]:
// test for matrix matrix multiplication using CUDA
std::vector<std::vector<double>> matrixA = {
        {1.0, 2.0},
        {3.0, 4.0},
        {5.0, 6.0}
    };

    std::vector<std::vector<double>> matrixB = {
        {1.0, 2.0, 3.0, 4.0},
        {5.0, 6.0, 7.0, 8.0}
    };

    std::vector<std::vector<double>> result = matmulCUDA(matrixA, matrixB);

    // Print the result
    std::cout << "Result matrix:" << std::endl;
    printMatrix(result);

Result matrix:
11 14 17 20 
23 30 37 44 
35 46 57 68 


In [18]:
//set up class and constructor
KalmanFilter::KalmanFilter(
    double dt,
    const std::vector<std::vector<double>>& A,
    const std::vector<std::vector<double>>& C,
    const std::vector<std::vector<double>>& Q,
    const std::vector<std::vector<double>>& R,
    const std::vector<std::vector<double>>& P)
  : A(A), C(C), Q(Q), R(R), P0(P),
    m(C.size()), n(A.size()), dt(dt), initialized(false),
    I(n, std::vector<double>(n)), x_hat(n), x_hat_new(n)
{
    for (int i = 0; i < n; i++) {
        I[i][i] = 1.0;
    }
}

KalmanFilter::KalmanFilter() {}

In [19]:
// init the kf states + measurements 
void KalmanFilter::init(double t0, const std::vector<double>& x0) {
    x_hat = x0;
    P = P0;
    this->t0 = t0;
    t = t0;
    initialized = true;
}

In [20]:
void KalmanFilter::init() {
    std::fill(x_hat.begin(), x_hat.end(), 0.0);
    P = P0;
    t0 = 0;
    t = t0;
    initialized = true;
}

In [21]:


void KalmanFilter::update(const std::vector<double>& y) {
    if (!initialized)
        throw std::runtime_error("Filter is not initialized!");

    x_hat_new = matvecmulCUDA(A, x_hat);

    P = mataddCUDA(matmulCUDA(matmulCUDA(A, P), mattransposeCUDA(A)), Q);

    std::vector<std::vector<double>> inv = matinverse(mataddCUDA(matmulCUDA(matmulCUDA(C, P), mattransposeCUDA(C)), R));

    K = matmulCUDA(matmulCUDA(P, mattransposeCUDA(C)), inv);
    std::vector<double> temp = matvecmulCUDA(C, x_hat_new);
    std::vector<double> difference = vecsubCUDA(y, temp);

    for (size_t i = 0; i < x_hat_new.size(); i++) {
        x_hat_new[i] += matvecmulCUDA(K, difference)[i];
    }

    P = matmulCUDA(matsubCUDA(I, matmulCUDA(K, C)), P);

    x_hat = x_hat_new;
    t += dt;
}


In [22]:
void KalmanFilter::update(const std::vector<double>& y, double dt, const std::vector<std::vector<double>>& A) {
    this->A = A;
    this->dt = dt;
    update(y);
}


In [23]:
std::vector<double> generateExpandedData(const std::vector<double>& baseData, int repetitions) {
    std::vector<double> expandedData;
    for (int r = 0; r < repetitions; r++) {
        for (auto val : baseData) {
            // Add slight random perturbation to the data.
            double perturbedValue = val + ((double)rand() / RAND_MAX - 0.5);
            expandedData.push_back(perturbedValue);
        }
    }
    return expandedData;
}

In [24]:
int run_kf(int expansion_factor, bool verbose) {
    
    int n = 3; // Number of states
    int m = 1; // Number of measurements

    double dt = 1.0 / 30; // Time step
    
    std::vector<std::vector<double>> A(n, std::vector<double>(n));
    std::vector<std::vector<double>> C(m, std::vector<double>(n));
    std::vector<std::vector<double>> Q(n, std::vector<double>(n));
    std::vector<std::vector<double>> R(m, std::vector<double>(m));
    std::vector<std::vector<double>> P(n, std::vector<double>(n));
    
    A = {{1, dt, 0}, {0, 1, dt}, {0, 0, 1}};
    C = {{1, 0, 0}};
    Q = {{.05, .05, .0}, {.05, .05, .0}, {.0, .0, .0}};
    R = {{5}};
    P = {{.1, .1, .1}, {.1, 10000, 10}, {.1, 10, 100}};
    
    KalmanFilter kf(dt, A, C, Q, R, P);
    
    std::vector<double> measurements = {
      1.04202710058, 1.10726790452, 1.2913511148, 1.48485250951, 1.72825901034,
      1.74216489744, 2.11672039768, 2.14529225112, 2.16029641405, 2.21269371128,
      2.57709350237, 2.6682215744, 2.51641839428, 2.76034056782, 2.88131780617,
      2.88373786518, 2.9448468727, 2.82866600131, 3.0006601946, 3.12920591669,
      2.858361783, 2.83808170354, 2.68975330958, 2.66533185589, 2.81613499531,
      2.81003612051, 2.88321849354, 2.69789264832, 2.4342229249, 2.23464791825,
      2.30278776224, 2.02069770395, 1.94393985809, 1.82498398739, 1.52526230354,
      1.86967808173, 1.18073207847, 1.10729605087, 0.916168349913, 0.678547664519,
      0.562381751596, 0.355468474885, -0.155607486619, -0.287198661013, -0.602973173813
      };
    
    std::vector<double> expandedMeasurements = generateExpandedData(measurements, expansion_factor);
    
    std::vector<double> x0 = {expandedMeasurements[0], 0, -9.81};
    kf.init(0, x0);

    // Feed measurements into filter, output estimated states
    std::vector<double> y(m);
    if(verbose) {
        std::cout << "t = " << 0 << ", " << "x_hat[0]: ";
        for (auto& val : kf.state()) std::cout << val << " ";
        std::cout << std::endl;
    }
    
    int i;
    for (i = 0; i < measurements.size(); i++) {
        y[0] = measurements[i];
        kf.update(y);
        if(verbose) {
            std::cout << "t = " << (i + 1) * dt << ", y[" << i << "] = " << y[0] << ", x_hat[" << i << "] = ";
            for (auto& val : kf.state()) std::cout << val << " ";
            std::cout << std::endl;
        }
    }
    std::cout << std::endl;
    std::cout<<"Exec Success, Final kf states:";
    for (auto& val : kf.state()) std::cout << val << " ";
    std::cout << std::endl;
    return 0;
}

In [25]:
#include <chrono>
auto start = std::chrono::high_resolution_clock::now();
auto stop = std::chrono::high_resolution_clock::now();
auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(stop - start);

In [26]:
int pyrun_sim(int exp_factor, bool verbose) {
    start = std::chrono::high_resolution_clock::now();
    run_kf(exp_factor, verbose);
    stop = std::chrono::high_resolution_clock::now();
    duration = std::chrono::duration_cast<std::chrono::milliseconds>(stop - start);
    return duration.count();
}

In [27]:
pyrun_sim(10, true)

t = 0, x_hat[0]: 0.737686 0 -9.81 
t = 0.0333333, y[0] = 1.04203, x_hat[0] = 0.948486 5.91215 -9.80189 
t = 0.0666667, y[1] = 1.10727, x_hat[1] = 1.11742 5.16313 -9.80246 
t = 0.1, y[2] = 1.29135, x_hat[2] = 1.29067 4.8479 -9.80243 
t = 0.133333, y[3] = 1.48485, x_hat[3] = 1.46956 4.65232 -9.8015 
t = 0.166667, y[4] = 1.72826, x_hat[4] = 1.67214 4.61508 -9.79665 
t = 0.2, y[5] = 1.74216, x_hat[5] = 1.79217 4.11586 -9.80259 
t = 0.233333, y[6] = 2.11672, x_hat[6] = 1.99697 4.08721 -9.78381 
t = 0.266667, y[7] = 2.14529, x_hat[7] = 2.13716 3.7765 -9.78219 
t = 0.3, y[8] = 2.1603, x_hat[8] = 2.23217 3.34189 -9.80002 
t = 0.333333, y[9] = 2.21269, x_hat[9] = 2.30708 2.89745 -9.82839 
t = 0.366667, y[10] = 2.57709, x_hat[10] = 2.44897 2.70633 -9.78267 
t = 0.4, y[11] = 2.66822, x_hat[11] = 2.57104 2.47129 -9.74221 
t = 0.433333, y[12] = 2.51642, x_hat[12] = 2.62117 2.05785 -9.79233 
t = 0.466667, y[13] = 2.76034, x_hat[13] = 2.70573 1.77429 -9.76272 
t = 0.5, y[14] = 2.88132, x_hat[14] = 2.

In [28]:
%%python

expand_factor = [500000, 1000000, 10000000, 20000000, 40000000]
benchmarks = []

for i in expand_factor:
    time_kf = cppyy.gbl.pyrun_sim(int(i), 0)
    benchmarks.append(time_kf)



Exec Success, Final kf states:-0.485139 -7.98525 -9.02124 

Exec Success, Final kf states:-0.436986 -7.62488 -8.35466 

Exec Success, Final kf states:-0.516416 -8.21932 -9.45422 

Exec Success, Final kf states:-0.476838 -7.92312 -8.90633 

Exec Success, Final kf states:-0.547245 -8.45003 -9.88098 


In [29]:
%%python

print(benchmarks)

[880, 1584, 13726, 27178, 54014]
